# 02a · Cleaning – Lehrvertragsauflösungen

**Zweck**  
Rohdaten in ein einheitliches, analysierbares Format bringen.

**Wichtigste Schritte**  
1. Alle *_Data-Sheets laden, fehlende Spalten harmonisieren  
2. Flags bauen  
   - `is_lva` ↔ mind. eine LVA-Zahl oder Suppression-Marker  
   - `is_wiedereinstieg` ↔ Wiedereinstiegs-Spalte nicht NA  
3. Kontextspalten ergänzen (`aggregation_level`, `kohorte_id`)  
4. Dubletten­killer auf definierter Key-Kombi  
5. Export: `tmp/lva_clean.parquet`

**Ergebnis**  
`tmp/lva_clean.parquet` – bereinigter, harmonisierter Datensatz.

In [1]:

# %%
# 02a_clean_lva.ipynb  –  Cleaning & Parquet
import pandas as pd
from pathlib import Path
from IPython.display import display

DATA_DIR  = Path("../../data")
TMP_DIR   = Path("../../tmp")
SRC_LVA   = DATA_DIR / "bfs_data_lva.xlsx"
TMP_FILE  = TMP_DIR  / "lva_clean.parquet"
TMP_DIR.mkdir(parents=True, exist_ok=True)

NUM_COLS  = ["anzahl_lernende", "anzahl_lehrvertraege_lva",
             "anzahl_lernende_lva", "anzahl_lernende_wiedereinstieg"]

KEEP_COLS = [
    "aggregation_level", "kohorte_id",
    "abschlussniveau", "lernform", "geschlecht", "mig_status",
    "lva_anschlussart", "qv_status", "lva_zeitpunkt", "wiedereinst_dauer",
    "ausbildungsfeld_isced_code", "ausbildungsfeld_isced_bez", "beruf_bez",
    *NUM_COLS,                      # ← fügt alle vier numerischen Felder ein
    "is_lva", "is_wiedereinstieg", "datenstatus"
]

def header_row(xls, sh):
    top = pd.read_excel(xls, sheet_name=sh, nrows=15, header=None)
    return next(i for i, r in top.iterrows() if r.notna().sum() >= 3)

print("Quelle:", SRC_LVA)

# ------------------------------------------------------------
# 1) Alle *_Data-Sheets durchgehen (einlesen)
# ------------------------------------------------------------
xls       = pd.ExcelFile(SRC_LVA)
DATA_SHEETS = [s for s in xls.sheet_names if s.endswith("_Data")]
all_frames  = []                         # statt rows-Liste



for sh in DATA_SHEETS:
    agg = sh.split("_")[0]                 # z. B. "T1"
    hdr = header_row(xls, sh)
    df  = pd.read_excel(xls, sheet_name=sh, header=hdr)

    # --------------------------------------------------
    # 1) fehlende numerische Spalten ergänzen
    # --------------------------------------------------

    for col in NUM_COLS:
        if col not in df.columns:
            df[col] = pd.NA

    # 2) fehlende kategoriale Spalte ergänzen
    if "datenstatus" not in df.columns:
        df["datenstatus"] = pd.NA

    # --------------------------------------------------
    # 3) Flags bauen & berechnen  (robust) - Sobald in irgendeiner der vier Spalten,inkl. datenstatus (etwa „<20“, „<30“) ein Wert steht, is_lva = 1.
    # --------------------------------------------------
    flag_cols = [
        "anzahl_lernende_lva",
        "anzahl_lehrvertraege_lva",
        "anzahl_lernende_wiedereinstieg",
        "datenstatus",
    ]
    df["is_lva"] = (
        df[df.columns.intersection(flag_cols)]
        .notna()
        .any(axis=1)
        .astype(int)
    )
    
    df["is_wiedereinstieg"] = (
        df["anzahl_lernende_wiedereinstieg"].notna() |
        df["datenstatus"].astype(str).str.startswith("<")
    ).astype(int)


    # 4) Kontextspalten ergänzen
    df["aggregation_level"] = agg
    df["kohorte_id"]        = 1

   # 5) genau die Zielspalten – fehlende werden automatisch angelegt
    df = df.reindex(columns=KEEP_COLS, fill_value=pd.NA)

    # 6) ans Gesamtergebnis anhängen
    all_frames.append(df)
    print(f"✓ {sh}: {len(df)} Zeilen")


# ------------------------------------------------------------
# 6) Gesamt-DataFrame, Dubletten killen
# ------------------------------------------------------------
df_all = pd.concat(all_frames, ignore_index=True)
before = len(df_all)
df_all = df_all.drop_duplicates(subset=[
    "aggregation_level","kohorte_id",
    "abschlussniveau","lernform","geschlecht","mig_status",
    "lva_anschlussart","qv_status","lva_zeitpunkt","wiedereinst_dauer",
    "ausbildungsfeld_isced_code","beruf_bez"
])
print(f"Dubletten entfernt: {before} → {len(df_all)}")

# ------------------------------------------------------------
# 7) Parquet schreiben
# ------------------------------------------------------------
df_all.to_parquet(TMP_FILE, index=False)
print("Parquet geschrieben:", TMP_FILE, "| Zeilen:", len(df_all))


Quelle: ..\..\data\bfs_data_lva.xlsx
✓ T1_Lernform_Data: 6 Zeilen
✓ T2_Geschlecht_Data: 9 Zeilen
✓ T3_MIG_Status_Data: 12 Zeilen
✓ T4.1.1_ISCED_Beruf_EBA_Data: 55 Zeilen
✓ T4.2.1_ISCED_Beruf_EFZ3_Data: 106 Zeilen
✓ T4.3.1_ISCED_Beruf_EFZ4_Data: 67 Zeilen
✓ T5_LVA_t_Data: 12 Zeilen
✓ T6_Wiedereinstieg_Data: 6 Zeilen
✓ T7_Zeitpkt_Wiedereinstieg_Data: 9 Zeilen
✓ T8_Geschlecht_Wiedereinst_Data: 6 Zeilen
✓ T9_ISCED_Beruf_WEstg_G_Data: 207 Zeilen
✓ T9_ISCED_Beruf_WEstg_EBA_Data: 48 Zeilen
✓ T9_ISCED_Beruf_WEstg_EFZ3_Data: 98 Zeilen
✓ T9_ISCED_Beruf_WEstg_EFZ4_Data: 61 Zeilen
✓ T10_Anschlussart_LVA_Data: 24 Zeilen
✓ T11_QV_Status_Ende_t_Data: 12 Zeilen
✓ T12_QV_Status_Ende_t_sex_Data: 24 Zeilen
✓ T13_QV_Status_Ende_t_MIG_Data: 48 Zeilen
✓ T14_QV_Status_Ende_t_Beruf_Data: 280 Zeilen
Dubletten entfernt: 1090 → 1081
Parquet geschrieben: ..\..\tmp\lva_clean.parquet | Zeilen: 1081


C:\Users\claud\AppData\Local\Temp\ipykernel_29264\2058143933.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat(all_frames, ignore_index=True)
